In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time

En este ejemplo, usaremos la librería de acceso libre Nominatim de OpenStreetMaps. No obstante, no siempre es fiable y la recomendad es GoogleV3

`from geopy.geocoders import GoogleV3`

`geolocator = GoogleV3(api_key=***)`

In [ ]:
def obtener_coordenadas(address):
    geolocator = Nominatim(user_agent="Tu-App/1.0")
    location = geolocator.geocode(address)
    time.sleep(1)               # Pausa de 1 segundo para evitar denegación de servicio
    if location:
        return location.latitude, location.longitude
    else:
        return None, None


In [3]:
df = pd.read_csv('../data/01_raw/211642-1-bomberos-parques.csv', sep=';', usecols=(9,10,12))
df.head()

,NOMBRE-VIA,CLASE-VIAL,NUM
0,SANTA ENGRACIA,CALLE,118
1,RUFINO BLANCO,CALLE,2
2,SEGOVIA,RONDA,95
3,EMILIA,CALLE,40
4,SANTA MARIA DE LA CABEZA,PASEO,95


In [ ]:
df['direccion'] = df.apply(lambda x: str(x['CLASE-VIAL'] + ' ' + x['NOMBRE-VIA']) + ', ' + str(x.NUM) + ', Madrid, Spain', axis=1)
df['coordenadas'] = df['direccion'].apply(obtener_coordenadas)
df

In [13]:
#Guardo las coordenadas en un array:
coord_riesgos = np.asarray(df.coordenadas)

### Grafico con folium

In [42]:
from folium import Map, plugins
from folium.plugins import HeatMap
map_madrid = Map(location=[40.42, -3.70], zoom_start = 12)

In [43]:
coord_riesgos = np.asarray(df.coordenadas)
HeatMap(coord_riesgos, radius=20).add_to(map_madrid)
map_madrid